In [ ]:
"""
HomeGuard Security System Simulator
Author: Elsa Bakiu
Description: A smart home monitoring system that processes sensor readings
             and triggers alerts for security, safety, and comfort issues.
"""

import random
from datetime import datetime

# System configuration
HOME_MODES = ["HOME", "AWAY", "SLEEP"]
ALERT_SEVERITIES = ["LOW", "MEDIUM", "HIGH", "CRITICAL"]

# Current system state
current_mode = "AWAY"


In [9]:
def create_sensor(sensor_id, location, sensor_type, threshold=None):
    """
    Creates a sensor data structure.
    
    Parameters:
    - sensor_id: Unique identifier for the sensor
    - location: Where the sensor is located (e.g., "Living Room")
    - sensor_type: Type of sensor ("motion", "temperature", "door", "smoke")
    - threshold: Optional threshold value for the sensor
    
    Returns:
    - A dictionary representing the sensor
    """
    sensor = {
        "id": sensor_id,
        "location": location,
        "type": sensor_type,
        "current_value": None,
        "threshold": threshold,
        "status": "OK"
    }
    return sensor
 
def create_alert(severity, message, sensor_id, timestamp):
    """
    Creates an alert data structure.
    
    Parameters:
    - severity: Alert severity level (LOW, MEDIUM, HIGH, CRITICAL)
    - message: Description of the alert
    - sensor_id: ID of the sensor that triggered the alert
    - timestamp: When the alert was triggered
    
    Returns:
    - A dictionary representing the alert
    """
    alert = {
        "severity": severity,
        "message": message,
        "sensor_id": sensor_id,
        "timestamp": timestamp
    }
    return alert
 
# Initialize sensors for the Peterson home
sensors = [
    create_sensor("S1", "Living Room", "motion"),
    create_sensor("S2", "Kitchen", "temperature", threshold=30),
    create_sensor("S3", "Front Door", "door"),
    create_sensor("S4", "Bedroom", "smoke")
]


In [10]:
print(f"Initialized {len(sensors)} sensors")
for sensor in sensors:
    print(f"  - {sensor['id']}: {sensor['location']} ({sensor['type']})")

Initialized 4 sensors
  - S1: Living Room (motion)
  - S2: Kitchen (temperature)
  - S3: Front Door (door)
  - S4: Bedroom (smoke)


In [12]:
def is_abnormal_reading(sensor, reading_value):
    """
    Checks if a sensor reading is abnormal based on sensor type and thresholds.
    
    Parameters:
    - sensor: Sensor dictionary
    - reading_value: The current reading from the sensor
    
    Returns:
    - True if the reading is abnormal, False otherwise
    """
    sensor_type = sensor["type"]
    
    # Temperature sensor: frozen pipe or overheating risk
    if sensor_type == "temperature":
        if reading_value < 35 or reading_value > 95:
            return True
        else:
            return False

    # Motion sensor: any detected motion is abnormal
    elif sensor_type == "motion":
        if reading_value is True:
            return True
        else:
            return False

    # Door sensor: door opened
    elif sensor_type == "door":
        if reading_value == "OPEN":
            return True
        else:
            return False

    # Smoke sensor: smoke detected
    elif sensor_type == "smoke":
        if reading_value == "DETECTED":
            return True
        else:
            return False

    # Default case: unknown sensor types are treated as normal
    else:
        return False


def should_trigger_security_alert(sensor, reading_value, system_mode):
    """
    Determines if a security alert should be triggered.
    
    Parameters:
    - sensor: Sensor dictionary
    - reading_value: The current reading from the sensor
    - system_mode: Current system mode (HOME, AWAY, SLEEP)
    
    Returns:
    - True if a security alert should be triggered, False otherwise
    """
    sensor_type = sensor["type"]

    # SMOKE: always urgent, regardless of mode
    if sensor_type == "smoke":
        return reading_value == "DETECTED"

    # TEMPERATURE: safety thresholds always trigger, regardless of mode
    if sensor_type == "temperature":
        # Safety alerts
        if reading_value < 35 or reading_value > 95:
            return True

        # Comfort notification only when HOME (outside 65-75°F)
        if system_mode == "HOME" and (reading_value < 65 or reading_value > 75):
            return True

        return False

    # MOTION: security-related, only meaningful when not HOME
    if sensor_type == "motion":
        if (system_mode == "AWAY" or system_mode == "SLEEP") and reading_value is True:
            return True
        return False

    # DOOR: security-related, only meaningful when not HOME
    if sensor_type == "door":
        if (system_mode == "AWAY" or system_mode == "SLEEP") and reading_value == "OPEN":
            return True
        return False

    # Unknown sensor type
    return False


In [13]:
# Test temperature check
test_sensor = create_sensor("TEMP_TEST", "Test Room", "temperature", threshold=35)
print(f"34°F is abnormal: {is_abnormal_reading(test_sensor, 34)}")  # Should be True
print(f"68°F is abnormal: {is_abnormal_reading(test_sensor, 68)}")  # Should be False

# Test security alert
motion_sensor = create_sensor("MOTION_TEST", "Test Room", "motion")
print(f"Motion in AWAY mode triggers alert: {should_trigger_security_alert(motion_sensor, True, 'AWAY')}")  # Should be True
print(f"Motion in HOME mode triggers alert: {should_trigger_security_alert(motion_sensor, True, 'HOME')}")  # Should be False


34°F is abnormal: True
68°F is abnormal: False
Motion in AWAY mode triggers alert: True
Motion in HOME mode triggers alert: False


In [14]:
import random
from datetime import datetime


def generate_reading(sensor):
    """
    Generates a realistic reading for a sensor.
    
    Parameters:
    - sensor: Sensor dictionary
    
    Returns:
    - A realistic reading value based on sensor type
    """    
    sensor_type = sensor["type"]

    if sensor_type == "temperature":
        return random.randint(30, 100)

    elif sensor_type == "motion":
        return random.choice([True, False])

    elif sensor_type == "door":
        return random.choice(["OPEN", "CLOSED"])

    elif sensor_type == "smoke":
        return random.choice(["CLEAR", "DETECTED"])

    else:
        return None


def process_reading(sensor, reading_value, system_mode):
    """
    Processes a sensor reading and determines if alerts are needed.
    
    Parameters:
    - sensor: Sensor dictionary
    - reading_value: The reading from the sensor
    - system_mode: Current system mode
    
    Returns:
    - A list of alerts (empty if no alerts needed)
    """
    alerts = []
    timestamp = datetime.now().strftime("%H:%M:%S")
    sensor_type = sensor["type"]
    sensor_id = sensor["id"]

    # Log the raw sensor reading
    log_event(
        f"Sensor {sensor_id} ({sensor_type}) reading: {reading_value}",
        timestamp
    )

    # 1. SECURITY ALERTS (motion / door in AWAY or SLEEP mode)
    if should_trigger_security_alert(sensor, reading_value, system_mode):
        alert = create_alert(
            severity="HIGH",
            message=f"Security alert from {sensor_type} sensor in {sensor['location']}",
            sensor_id=sensor_id,
            timestamp=timestamp
        )
        alerts.append(alert)

    # 2. SAFETY ALERTS (temperature extremes, smoke)
    if sensor_type == "temperature":
        if reading_value < 35:
            alerts.append(create_alert(
                severity="CRITICAL",
                message="Temperature below 35°F (frozen pipe risk)",
                sensor_id=sensor_id,
                timestamp=timestamp
            ))
        elif reading_value > 95:
            alerts.append(create_alert(
                severity="CRITICAL",
                message="Temperature above 95°F (fire or equipment risk)",
                sensor_id=sensor_id,
                timestamp=timestamp
            ))

    if sensor_type == "smoke" and reading_value == "DETECTED":
        alerts.append(create_alert(
            severity="CRITICAL",
            message="Smoke detected! Possible fire hazard",
            sensor_id=sensor_id,
            timestamp=timestamp
        ))

    # 3. COMFORT NOTIFICATIONS (HOME mode only)
    if sensor_type == "temperature" and system_mode == "HOME":
        if reading_value < 65 or reading_value > 75:
            alerts.append(create_alert(
                severity="LOW",
                message="Temperature outside comfort range (65–75°F)",
                sensor_id=sensor_id,
                timestamp=timestamp
            ))

    return alerts


def trigger_alert(alert):
    """
    Displays an alert to the user.
    
    Parameters:
    - alert: Alert dictionary
    """
    severity_symbol = {
        "LOW": "ℹ️",
        "MEDIUM": "⚠️",
        "HIGH": "🚨",
        "CRITICAL": "🔥"
    }
    
    symbol = severity_symbol.get(alert["severity"], "⚠️")
    print(f"[ALERT!] {symbol} {alert['severity']}: {alert['message']}")


def log_event(message, timestamp=None):
    """
    Logs an event to the console.
    
    Parameters:
    - message: The message to log
    - timestamp: Optional timestamp (uses current time if not provided)
    """
    if timestamp is None:
        timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[LOG] [{timestamp}] {message}")

In [15]:
# Test reading generation
test_sensor = sensors[0]  # Motion sensor
reading = generate_reading(test_sensor)
print(f"Generated reading for {test_sensor['location']}: {reading}")

# Test processing
alerts = process_reading(test_sensor, True, "AWAY")
if alerts:
    trigger_alert(alerts[0])


Generated reading for Living Room: True
[LOG] [21:49:22] Sensor S1 (motion) reading: True
[ALERT!] 🚨 HIGH: Security alert from motion sensor in Living Room


In [16]:
import random


class Sensor:
    """
    Represents a sensor in the HomeGuard system.
    """
    
    def __init__(self, sensor_id, location, sensor_type, threshold=None):
        """
        Initializes a new sensor.
        
        Parameters:
        - sensor_id: Unique identifier for the sensor
        - location: Where the sensor is located
        - sensor_type: Type of sensor ("motion", "temperature", "door", "smoke")
        - threshold: Optional threshold value for the sensor
        """
        self.id = sensor_id
        self.location = location
        self.type = sensor_type
        self.threshold = threshold
        self.current_value = None
 
    
    def read(self):
        """
        Generates and stores a new reading for this sensor.
        
        Returns:
        - The reading value
        """
        if self.type == "temperature":
            self.current_value = random.randint(30, 100)

        elif self.type == "motion":
            self.current_value = random.choice([True, False])

        elif self.type == "door":
            self.current_value = random.choice(["OPEN", "CLOSED"])

        elif self.type == "smoke":
            self.current_value = random.choice(["CLEAR", "DETECTED"])

        else:
            self.current_value = None

        return self.current_value
        
    
    def isAbnormal(self):
        """
        Checks if the current reading is abnormal.
        
        Returns:
        - True if the reading is abnormal, False otherwise
        """
        if self.current_value is None:
            return False

        # Temperature sensor
        if self.type == "temperature":
            return self.current_value < 35 or self.current_value > 95

        # Motion sensor
        if self.type == "motion":
            return self.current_value is True

        # Door sensor
        if self.type == "door":
            return self.current_value == "OPEN"

        # Smoke sensor
        if self.type == "smoke":
            return self.current_value == "DETECTED"

        return False
  
    
    def reset(self):
        """
        Resets the sensor's current reading to None.
        """
        self.current_value = None
    
    def __str__(self):
        """
        Returns a string representation of the sensor.
        """
        status = "No reading" if self.current_value is None else str(self.current_value)
        return f"{self.id} ({self.location}): {status}"
 

# Create sensor objects using the class
sensor_objects = [
    Sensor("MOTION_001", "Living Room", "motion"),
    Sensor("TEMP_001", "Kitchen", "temperature", threshold=35),
    Sensor("DOOR_001", "Front Door", "door"),
    Sensor("SMOKE_001", "Bedroom", "smoke")
]


In [ ]:
# Create and test a sensor
test_sensor = Sensor("TEST_001", "Test Room", "temperature", threshold=35)
test_sensor.read()
print(f"Sensor reading: {test_sensor.current_value}")
print(f"Is abnormal: {test_sensor.isAbnormal()}")
print(f"Sensor info: {test_sensor}")


Sensor reading: 75
Is abnormal: False
Sensor info: TEST_001 (Test Room): 75


In [20]:
import random
import time
from datetime import datetime


def create_alert(severity, message, sensor_id, timestamp):
    """
    Creates an alert data structure.
    
    Parameters:
    - severity: Alert severity level (LOW, MEDIUM, HIGH, CRITICAL)
    - message: Description of the alert
    - sensor_id: ID of the sensor that triggered the alert
    - timestamp: When the alert was triggered
    
    Returns:
    - A dictionary representing the alert
    """
    return {
        "severity": severity,
        "message": message,
        "sensor_id": sensor_id,
        "timestamp": timestamp
    }


def log_event(message, timestamp=None):
    """
    Logs an event to the console.
    
    Parameters:
    - message: The message to log
    - timestamp: Optional timestamp (uses current time if not provided)
    """
    if timestamp is None:
        timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[LOG] [{timestamp}] {message}")


def trigger_alert(alert):
    """
    Displays an alert to the user.
    
    Parameters:
    - alert: Alert dictionary
    """
    severity_symbol = {
        "LOW": "ℹ️",
        "MEDIUM": "⚠️",
        "HIGH": "🚨",
        "CRITICAL": "🔥"
    }
    
    symbol = severity_symbol.get(alert["severity"], "⚠️")
    print(f"[ALERT!] {symbol} {alert['severity']}: {alert['message']}")


def should_trigger_security_alert(sensor, reading_value, system_mode):
    """
    Determines if an alert should be triggered based on sensor type, reading, and system mode.

    Note: The function name comes from an earlier step, but this version supports
    all sensor types (motion, door, temperature, smoke) and mode-based behavior.

    Parameters:
    - sensor: Sensor dictionary
    - reading_value: The current reading from the sensor
    - system_mode: Current system mode (HOME, AWAY, SLEEP)

    Returns:
    - True if an alert/notification should be triggered, False otherwise
    """
    sensor_type = sensor["type"]

    # SMOKE: always urgent, regardless of mode
    if sensor_type == "smoke":
        return reading_value == "DETECTED"

    # TEMPERATURE: safety thresholds always trigger, regardless of mode
    if sensor_type == "temperature":
        if reading_value < 35 or reading_value > 95:
            return True
        if system_mode == "HOME" and (reading_value < 65 or reading_value > 75):
            return True
        return False

    # MOTION: security-related, only meaningful when not HOME
    if sensor_type == "motion":
        return (system_mode in ["AWAY", "SLEEP"]) and (reading_value is True)

    # DOOR: security-related, only meaningful when not HOME
    if sensor_type == "door":
        return (system_mode in ["AWAY", "SLEEP"]) and (reading_value == "OPEN")

    return False


def process_reading(sensor, reading_value, system_mode):
    """
    Processes a sensor reading and determines if alerts are needed.
    
    Parameters:
    - sensor: Sensor dictionary
    - reading_value: The reading from the sensor
    - system_mode: Current system mode
    
    Returns:
    - A list of alerts (empty if no alerts needed)
    """
    alerts = []
    timestamp = datetime.now().strftime("%H:%M:%S")
    sensor_type = sensor["type"]
    sensor_id = sensor["id"]

    # Log every reading
    log_event(f"Sensor {sensor_id} ({sensor_type}) reading: {reading_value}", timestamp)

    # 1) Security alerts (motion/door in AWAY or SLEEP)
    if sensor_type in ["motion", "door"] and should_trigger_security_alert(sensor, reading_value, system_mode):
        message = f"{sensor_type.capitalize()} activity detected while system is {system_mode}"
        alerts.append(create_alert("HIGH", message, sensor_id, timestamp))

    # 2) Safety alerts
    if sensor_type == "temperature":
        if reading_value < 35:
            alerts.append(create_alert("CRITICAL", "Temperature below 35°F (frozen pipe risk)", sensor_id, timestamp))
        elif reading_value > 95:
            alerts.append(create_alert("CRITICAL", "Temperature above 95°F (equipment/fire risk)", sensor_id, timestamp))

    if sensor_type == "smoke" and reading_value == "DETECTED":
        alerts.append(create_alert("CRITICAL", "Smoke detected! Possible fire risk", sensor_id, timestamp))

    # 3) Comfort notifications (HOME mode only)
    if sensor_type == "temperature" and system_mode == "HOME":
        if reading_value < 65 or reading_value > 75:
            alerts.append(create_alert("LOW", "Temperature outside comfort range (65–75°F)", sensor_id, timestamp))

    return alerts


class Sensor:
    """
    Represents a sensor in the HomeGuard system.
    """
    
    def __init__(self, sensor_id, location, sensor_type, threshold=None):
        """
        Initializes a new sensor.
        
        Parameters:
        - sensor_id: Unique identifier for the sensor
        - location: Where the sensor is located
        - sensor_type: Type of sensor ("motion", "temperature", "door", "smoke")
        - threshold: Optional threshold value for the sensor
        """
        self.id = sensor_id
        self.location = location
        self.type = sensor_type
        self.threshold = threshold
        self.current_value = None

    def read(self):
        """
        Generates and stores a new reading for this sensor.
        
        Returns:
        - The reading value
        """
        if self.type == "temperature":
            self.current_value = random.randint(30, 100)
        elif self.type == "motion":
            self.current_value = random.choice([True, False])
        elif self.type == "door":
            self.current_value = random.choice(["OPEN", "CLOSED"])
        elif self.type == "smoke":
            self.current_value = random.choice(["CLEAR", "DETECTED"])
        else:
            self.current_value = None
        return self.current_value
  
    def isAbnormal(self):
        """
        Checks if the current reading is abnormal.
        
        Returns:
        - True if the reading is abnormal, False otherwise
        """
        if self.current_value is None:
            return False

        if self.type == "temperature":
            return self.current_value < 35 or self.current_value > 95
        if self.type == "motion":
            return self.current_value is True
        if self.type == "door":
            return self.current_value == "OPEN"
        if self.type == "smoke":
            return self.current_value == "DETECTED"

        return False
    
    def reset(self):
        """
        Resets the sensor's current reading to None.
        """
        self.current_value = None
    
    def __str__(self):
        """
        Returns a string representation of the sensor.
        """
        status = "No reading" if self.current_value is None else str(self.current_value)
        return f"{self.id} ({self.location}): {status}"


def run_simulation(duration_minutes=5, system_mode="AWAY"):
    """
    Runs the HomeGuard security system simulation.
    
    Parameters:
    - duration_minutes: How long to run the simulation (default: 5 minutes)
    - system_mode: System mode (HOME, AWAY, SLEEP)
    """
    print("=" * 50)
    print("=== HomeGuard Security System ===")
    print("=" * 50)
    print(f"Mode: {system_mode}\n")
    
    # Use sensor objects instead of dictionaries
    sensors = [
        Sensor("MOTION_001", "Living Room", "motion"),
        Sensor("TEMP_001", "Kitchen", "temperature", threshold=35),
        Sensor("DOOR_001", "Front Door", "door"),
        Sensor("SMOKE_001", "Bedroom", "smoke")
    ]
    
    # Simulate time passing (each iteration = 1 minute)
    for minute in range(duration_minutes):
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"\nTime: {current_time}")
        
        # Read all sensors
        for sensor in sensors:
            reading = sensor.read()
            
            # Display the reading
            if sensor.type == "temperature":
                status = "Normal" if 65 <= reading <= 75 else "Abnormal"
                print(f"[READING] {sensor.location} Temperature: {reading}°F ({status})")
            elif sensor.type == "motion":
                status = "DETECTED" if reading else "No activity"
                print(f"[READING] {sensor.location} Motion: {status}")
            elif sensor.type == "door":
                print(f"[READING] {sensor.location}: {reading}")
            elif sensor.type == "smoke":
                print(f"[READING] {sensor.location} Smoke: {reading}")
            
            # Process the reading and trigger alerts if needed
            # Convert sensor object to dict format for process_reading function
            sensor_dict = {
                "id": sensor.id,
                "location": sensor.location,
                "type": sensor.type,
                "threshold": sensor.threshold
            }
            alerts = process_reading(sensor_dict, reading, system_mode)
            
            # Trigger all alerts
            for alert in alerts:
                trigger_alert(alert)
                if alert["severity"] in ["HIGH", "CRITICAL"]:
                    log_event("Sending notification to homeowner...")
        
        # Small delay to make output readable (optional)
        time.sleep(0.5)
    
    print("\n" + "=" * 50)
    print("Simulation complete!")
    print("=" * 50)
 

# Main execution
if __name__ == "__main__":
    # Run the simulation
    run_simulation(duration_minutes=3, system_mode="AWAY")



=== HomeGuard Security System ===
Mode: AWAY


Time: 22:00:40
[READING] Living Room Motion: No activity
[LOG] [22:00:40] Sensor MOTION_001 (motion) reading: False
[READING] Kitchen Temperature: 98°F (Abnormal)
[LOG] [22:00:40] Sensor TEMP_001 (temperature) reading: 98
[ALERT!] 🔥 CRITICAL: Temperature above 95°F (equipment/fire risk)
[LOG] [22:00:40] Sending notification to homeowner...
[READING] Front Door: CLOSED
[LOG] [22:00:40] Sensor DOOR_001 (door) reading: CLOSED
[READING] Bedroom Smoke: CLEAR
[LOG] [22:00:40] Sensor SMOKE_001 (smoke) reading: CLEAR

Time: 22:00:41
[READING] Living Room Motion: DETECTED
[LOG] [22:00:41] Sensor MOTION_001 (motion) reading: True
[ALERT!] 🚨 HIGH: Motion activity detected while system is AWAY
[LOG] [22:00:41] Sending notification to homeowner...
[READING] Kitchen Temperature: 52°F (Abnormal)
[LOG] [22:00:41] Sensor TEMP_001 (temperature) reading: 52
[READING] Front Door: OPEN
[LOG] [22:00:41] Sensor DOOR_001 (door) reading: OPEN
[ALERT!] 🚨 HIGH: Door

In [21]:
from datetime import datetime

# Helper: run exactly ONE simulated minute using forced readings (no randomness)
def run_forced_minute(system_mode, forced_readings):
    """
    Runs one simulation cycle using forced sensor readings so you can test exact scenarios.
    forced_readings is a dict like:
      {"MOTION_001": True, "TEMP_001": 72, "DOOR_001": "OPEN", "SMOKE_001": "CLEAR"}
    """
    sensors = [
        Sensor("MOTION_001", "Living Room", "motion"),
        Sensor("TEMP_001", "Kitchen", "temperature", threshold=35),
        Sensor("DOOR_001", "Front Door", "door"),
        Sensor("SMOKE_001", "Bedroom", "smoke")
    ]

    print("\n" + "=" * 50)
    print(f"FORCED TEST | Mode: {system_mode}")
    print(f"Time: {datetime.now().strftime('%H:%M:%S')}")
    print("=" * 50)

    for sensor in sensors:
        # Force the reading
        reading = forced_readings.get(sensor.id, None)
        sensor.current_value = reading  # store it on the object too

        # Display reading in the same format as your simulator
        if sensor.type == "temperature":
            status = "Normal" if 65 <= reading <= 75 else "Abnormal"
            print(f"[READING] {sensor.location} Temperature: {reading}°F ({status})")
        elif sensor.type == "motion":
            status = "DETECTED" if reading else "No activity"
            print(f"[READING] {sensor.location} Motion: {status}")
        elif sensor.type == "door":
            print(f"[READING] {sensor.location}: {reading}")
        elif sensor.type == "smoke":
            print(f"[READING] {sensor.location} Smoke: {reading}")

        # process_reading expects a dict (as in your template)
        sensor_dict = {
            "id": sensor.id,
            "location": sensor.location,
            "type": sensor.type,
            "threshold": sensor.threshold
        }

        alerts = process_reading(sensor_dict, reading, system_mode)

        for alert in alerts:
            trigger_alert(alert)
            if alert["severity"] in ["HIGH", "CRITICAL"]:
                log_event("Sending notification to homeowner...")

    print("=" * 50)


# =========================
# EXACT SCENARIOS YOU LISTED
# =========================

def security_test_away():
    """
    Security Test:
    Run with system_mode="AWAY" and verify security alerts trigger.
    Expect: HIGH alert for motion detected and HIGH alert for door opened.
    """
    run_forced_minute(
        system_mode="AWAY",
        forced_readings={
            "MOTION_001": True,        # should trigger HIGH security alert
            "TEMP_001": 72,            # normal
            "DOOR_001": "OPEN",        # should trigger HIGH security alert
            "SMOKE_001": "CLEAR"       # normal
        }
    )


def safety_test_extremes_and_smoke():
    """
    Safety Test:
    Check that temperature and smoke alerts work.
    Expect: CRITICAL alert for low temp, CRITICAL for high temp, CRITICAL for smoke detected.
    """
    # 1) Temperature below 35°F
    run_forced_minute(
        system_mode="HOME",
        forced_readings={
            "MOTION_001": False,
            "TEMP_001": 32,            # CRITICAL
            "DOOR_001": "CLOSED",
            "SMOKE_001": "CLEAR"
        }
    )

    # 2) Temperature above 95°F
    run_forced_minute(
        system_mode="AWAY",
        forced_readings={
            "MOTION_001": False,
            "TEMP_001": 98,            # CRITICAL
            "DOOR_001": "CLOSED",
            "SMOKE_001": "CLEAR"
        }
    )

    # 3) Smoke detected
    run_forced_minute(
        system_mode="SLEEP",
        forced_readings={
            "MOTION_001": False,
            "TEMP_001": 70,
            "DOOR_001": "CLOSED",
            "SMOKE_001": "DETECTED"    # CRITICAL
        }
    )


def comfort_test_home():
    """
    Comfort Test:
    Run with system_mode="HOME" and verify comfort notifications appear.
    Expect: LOW alert when temperature is outside 65-75 but not a safety extreme.
    """
    run_forced_minute(
        system_mode="HOME",
        forced_readings={
            "MOTION_001": False,
            "TEMP_001": 78,            # LOW comfort notification
            "DOOR_001": "CLOSED",
            "SMOKE_001": "CLEAR"
        }
    )


# Run the exact tests
if __name__ == "__main__":
    security_test_away()
    safety_test_extremes_and_smoke()
    comfort_test_home()



FORCED TEST | Mode: AWAY
Time: 22:00:58
[READING] Living Room Motion: DETECTED
[LOG] [22:00:58] Sensor MOTION_001 (motion) reading: True
[ALERT!] 🚨 HIGH: Motion activity detected while system is AWAY
[LOG] [22:00:58] Sending notification to homeowner...
[READING] Kitchen Temperature: 72°F (Normal)
[LOG] [22:00:58] Sensor TEMP_001 (temperature) reading: 72
[READING] Front Door: OPEN
[LOG] [22:00:58] Sensor DOOR_001 (door) reading: OPEN
[ALERT!] 🚨 HIGH: Door activity detected while system is AWAY
[LOG] [22:00:58] Sending notification to homeowner...
[READING] Bedroom Smoke: CLEAR
[LOG] [22:00:58] Sensor SMOKE_001 (smoke) reading: CLEAR

FORCED TEST | Mode: HOME
Time: 22:00:58
[READING] Living Room Motion: No activity
[LOG] [22:00:58] Sensor MOTION_001 (motion) reading: False
[READING] Kitchen Temperature: 32°F (Abnormal)
[LOG] [22:00:58] Sensor TEMP_001 (temperature) reading: 32
[ALERT!] 🔥 CRITICAL: Temperature below 35°F (frozen pipe risk)
[LOG] [22:00:58] Sending notification to homeo